In [4]:
import numpy as np
from pathlib import Path, PosixPath
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr

Path to dataset

In [5]:
path = PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS')

In [6]:
path

PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS')

# load characteristics 

In [5]:
indicator_files = list(path.glob('*.txt'))

In [6]:
df_clim_indicators = pd.read_csv(indicator_files[0],delimiter=";")
df_geol_indicators = pd.read_csv(indicator_files[1],delimiter=";")
df_hydro_indicators = pd.read_csv(indicator_files[2],delimiter=";")
df_catchment_names = pd.read_csv(indicator_files[3],delimiter=";")
df_soil_indicators = pd.read_csv(indicator_files[4],delimiter=";")
df_topo_indicators = pd.read_csv(indicator_files[5],delimiter=";")
df_vege_indicators = pd.read_csv(indicator_files[6],delimiter=";")
df_vege_indicators['dom_land_cover'] = df_vege_indicators['dom_land_cover'].apply(lambda x: x.strip())

In [7]:
df_char_lst = [df_geol_indicators,
                df_clim_indicators,
                df_hydro_indicators,
                df_catchment_names,
                df_soil_indicators,
                df_vege_indicators, 
                df_topo_indicators]
for index, df in enumerate(df_char_lst):
    df_char_lst[index] = df.set_index('gauge_id')

In [8]:
characteristics_all = pd.concat(df_char_lst,axis=1)

## Forcing path

In [9]:
source_lst = ['daymet','maurer','nldas']
source = source_lst[0]
forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
forcing_sub_path_2 = f'basin_dataset_public_v1p2'
forcing_sub_path_3 = f'basin_mean_forcing'

In [10]:
forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source

In [11]:
sub_dirs_forcing = list(forcing_path.glob("*"))

In [12]:
basin_mean_forcing_files = list(sub_dirs_forcing[0].glob("*.txt"))

In [13]:
basin_mean_forcing_file = basin_mean_forcing_files[0]
basin_id = basin_mean_forcing_file.name[:8]
folder_id = basin_mean_forcing_file.parent.name

In [14]:
catchment_char = characteristics_all.loc[int(basin_id)].copy()

In [15]:
len(list(characteristics_all.index))

671

## retrieve alpha

retrieve alpha: not always 1.26

In [15]:
model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'

In [16]:
model_ouput_path

PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS/basin_timeseries_v1p2_modelOutput_daymet/model_output_daymet/model_output/flow_timeseries/daymet')

In [17]:
parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))

In [18]:
line_start = "pet_coef"
lst_alpha = []
for file in parameter_files_path:
    with open(file,'r') as lines:
        for line in lines:
            if line[:len(line_start)] == line_start:
                lst_alpha.append(float(line.strip().split("\t")[-1]))

In [19]:
alpha = np.array(lst_alpha).mean()

## add streamflow

In [20]:
streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'
streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 

# Combine all 

In [21]:
from read_camels import from_camels_txt, crop_ds, calc_pet

In [22]:
start_date = "1980-01-01T00:00:00Z"
end_date = "2014-12-31T00:00:00Z"

In [23]:
out_path = path.cwd() / "Output" / source
out_path.mkdir(exist_ok=True)

In [24]:
ds0 = from_camels_txt(basin_mean_forcing_file, basin_id, source, start_date, end_date, out_path, alpha, catchment_char, streamflow_file_path)

## test merge

In [25]:
source_lst = ['daymet','maurer','nldas']
source = source_lst[0]
# Source dependant
forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
forcing_sub_path_2 = f'basin_dataset_public_v1p2'
forcing_sub_path_3 = f'basin_mean_forcing'
forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source
sub_dirs_forcing = list(forcing_path.glob("*"))
model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'
streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'

In [26]:
# basin dependant
basin_mean_forcing_file = basin_mean_forcing_files[1]
basin_id = basin_mean_forcing_file.name[:8]
folder_id = basin_mean_forcing_file.parent.name
catchment_char = characteristics_all.loc[int(basin_id)].copy()
parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))
line_start = "pet_coef"
lst_alpha = []
for file in parameter_files_path:
    with open(file,'r') as lines:
        for line in lines:
            if line[:len(line_start)] == line_start:
                lst_alpha.append(float(line.strip().split("\t")[-1]))
alpha = np.array(lst_alpha).mean()

streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 

In [27]:
ds1 = from_camels_txt(basin_mean_forcing_file, basin_id, source, start_date, end_date, out_path, alpha, catchment_char, streamflow_file_path)

In [28]:
xr.merge([ds0,ds1])

<xarray.Dataset> Size: 2MB
Dimensions:               (time: 12784, data_source: 1, basin_id: 2)
Coordinates:
  * time                  (time) datetime64[ns] 102kB 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 64B b'daymet'
  * basin_id              (basin_id) |S64 128B b'01013500' b'01134500'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 205kB 3.017e+...
    pr                    (basin_id, data_source, time) float64 205kB 0.0 ......
    srad                  (basin_id, data_source, time) float64 205kB 153.4 ....
    swe                   (basin_id, data_source, time) float64 205kB 0.0 ......
    tasmax                (basin_id, data_source, time) float64 205kB -6.54 ....
    tasmin                (basin_id, data_source, time) float64 205kB -16.3 ....
    ...                    ...
    gauge_lat             (basin_id) float64 16B 47.24 44.51
    gauge_lon             (basin_id) float64 16B -68.58 -71.84
    elev_mean             (basin_id) float64 16B 250.3 450.5
    slope_mean            (basin_id) float64 16B 21.64 47.54
    area_gages2           (basin_id) float64 16B 2.253e+03 195.1
    area_geospa_fabric    (basin_id) float64 16B 2.304e+03 209.5
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2617
    lat:              44.49
    elevation(m):     529.0
    area basin(m^2):  194683851.0

# loop

not found 1150900, 6775500, 6846500 in `catchment_char = characteristics_all.loc[int(basin_id)].copy()`

error in 02108000, 05120500, 07067000, 09492400 wrong header daymet

In [29]:
skip_ids = [2081113, 3448942, 9535100, 1150900, 6775500, 6846500]

In [30]:
df

,gauge_id,gauge_lat,gauge_lon,elev_mean,slope_mean,area_gages2,area_geospa_fabric
0,1013500,47.23739,-68.58264,250.31,21.64152,2252.70,2303.95
1,1022500,44.60797,-67.93524,92.68,17.79072,573.60,620.38
2,1030500,45.50097,-68.30596,143.80,12.79195,3676.17,3676.09
3,1031500,45.17501,-69.31470,247.80,29.56035,769.05,766.53
4,1047000,44.86920,-69.95510,310.38,49.92122,909.10,904.94
...,...,...,...,...,...,...,...
666,14309500,42.80400,-123.61091,709.83,110.42527,224.92,226.31
667,14316700,43.34984,-122.72894,952.26,119.08920,587.90,588.01
668,14325000,42.89150,-124.07065,656.53,124.96889,443.07,444.92
669,14362250,42.15401,-123.07532,875.67,109.93127,41.42,43.88


In [36]:
source_lst = ['daymet','maurer','nldas']
for source in source_lst[1:]:
    # Source dependant
    out_path = path.cwd() / "Output" / source
    out_path.mkdir(exist_ok=True)
    forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
    forcing_sub_path_2 = f'basin_dataset_public_v1p2'
    forcing_sub_path_3 = f'basin_mean_forcing'
    forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source
    sub_dirs_forcing = list(forcing_path.glob("*"))
    model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'
    streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'
    
    for sub_dirs in sub_dirs_forcing:
        basin_mean_forcing_files = list(sub_dirs.glob("*.txt"))
        for basin_mean_forcing_file in basin_mean_forcing_files:
            basin_id = basin_mean_forcing_file.name[:8]
            if int(basin_id) in skip_ids:
                pass 
            else:
                folder_id = basin_mean_forcing_file.parent.name
                catchment_char = characteristics_all.loc[int(basin_id)].copy()
                parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))
                
                if source == "daymet":
                    line_start = "pet_coef"
                elif source != 'daymet':
                    line_start = "PT_COEF"
                lst_alpha = []    
                for file in parameter_files_path:
                    with open(file,'r') as lines:
                        for line in lines:
                            if line[:len(line_start)] == line_start:
                                lst_alpha.append(float(line.strip().split("\t")[-1]))
                alpha = np.array(lst_alpha).mean()
                
                streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 
                ds = from_camels_txt(basin_mean_forcing_file, 
                                     basin_id, 
                                     source, 
                                     start_date, 
                                     end_date, 
                                     out_path, 
                                     alpha, 
                                     catchment_char, 
                                     streamflow_file_path)
                ds.close()

In [7]:
files_daymet = list((path.cwd() / "Output" / "daymet").glob("*nc"))
files_maurer = list(( path.cwd() / "Output" / "maurer").glob("*nc"))
files_nldas = list(( path.cwd() / "Output" / "nldas").glob("*nc"))

In [8]:
file_lst = []
names = files_daymet + files_maurer + files_nldas
for file in names[:2]:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file)
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [13]:
ds = xr.concat(file_lst,dim=('time'))

In [14]:
ds

<xarray.Dataset> Size: 28MB
Dimensions:               (basin_id: 2, data_source: 1, time: 25568)
Coordinates:
  * data_source           (data_source) |S64 64B b'daymet'
  * basin_id              (basin_id) object 16B b'camels_02395120' b'camels_0...
  * time                  (time) datetime64[ns] 205kB 1980-01-01 ... 2014-12-31
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 409kB 3.525e+...
    pr                    (basin_id, data_source, time) float64 409kB 5.0 ......
    srad                  (basin_id, data_source, time) float64 409kB 127.6 ....
    swe                   (basin_id, data_source, time) float64 409kB 0.0 ......
    tasmax                (basin_id, data_source, time) float64 409kB 6.5 ......
    tasmin                (basin_id, data_source, time) float64 409kB 1.0 ......
    ...                    ...
    gauge_lat             (time, basin_id) float64 409kB 34.24 nan ... nan 45.26
    gauge_lon             (time, basin_id) float64 409kB -84.89 nan ... -100.8
    elev_mean             (time, basin_id) float64 409kB 250.6 nan ... nan 802.5
    slope_mean            (time, basin_id) float64 409kB 9.609 nan ... nan 10.51
    area_gages2           (time, basin_id) float64 409kB 84.01 nan ... 1.265e+04
    area_geospa_fabric    (time, basin_id) float64 409kB 85.27 nan ... 1.26e+04
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2640999999999998
    lat:              34.24
    elevation(m):     264.0
    area basin(m^2):  85691961.0

In [ ]:
ds.to_netcdf("combined.nc")